In [1]:
import pandas as pd
import re
import jieba
#构建中文分词类v3.0
class Leijieba:    
    
    __stopwords = ''
        
    def __init__(self,s,t='',b=''):    #t为停用词的文件参数（路径+文件名）；b为本地词库的文件参数（路径+文件名）
        self.__s = s
        
        if t != '':
            self.__stopwords = [line.strip() for line in open(t,encoding='UTF-8').readlines()]  #逐行读取停用词文件，放入类内部变量__stopwords
            
        if b != '':
            jieba.load_userdict(b)
        
    def cut(self):
        seg_list = jieba.cut(self.__s, cut_all=False)
        if self.__stopwords != '':
            outstr = []    #目的是存储去掉停用用词的分词结果
            for word in list(seg_list):
                if word not in self.__stopwords:
                    if word.strip() !='':   #去空格
                        outstr.append(word)
                        
            seg_list = outstr                                                
        return list(seg_list)
    
    def num(self):
        lst = self.cut()
        
        word_dict = {}        
        for item in lst:
            if item not in word_dict:
                word_dict[item] = 1   #新建“键-值”
            else:
                word_dict[item] += 1  #增加 键的值
        return sorted(word_dict.items(), key=lambda e:e[1], reverse=True)  
    
    def cut1(self,num = 0,sw = 0):   #1.删除词频低于lnum的词,2.if sw ==1,删除单字的词
        lst = self.cut()
        
        word_dict = {}        
        for item in lst:
            if item not in word_dict:
                word_dict[item] = 1   #新建“键-值”
            else:
                word_dict[item] += 1  #增加 键的值
        dic = dict(sorted(word_dict.items(), key=lambda e:e[1], reverse=True)) 
        lst1 = lst[:]
        
        if sw == 1:
            for item in lst:
                if len(item) == 1:
                    lst1.remove(item)
                else:
                    if dic[item] <= num:
                        lst1.remove(item)
        else:
            for item in lst:
                if dic[item] <= num:
                    lst1.remove(item)
                
        return lst1

In [2]:
data = pd.read_csv('./需要预测的论坛数据.csv')
data = data.iloc[110000:]
data = data.reset_index(drop=True)
data

,publish_time,view_counts,summarization
0,2024-07-17 09:27:20,54,黄金建议2477空止损2484目标2465 2457 2450
1,2024-07-17 09:38:47,163,出不去的资金都没有更好的投资方式黄金与四大行的股票是仅剩的保值方向
2,2024-07-17 09:45:26,257,昨晚梦中刨土从土里刨出了孙猴子梦醒看一眼时间0219再看一眼黄金走势联想起刚刚的梦境孙悟空空...
3,2024-07-17 09:53:46,184,黄金真是给力周一还在2401今天就到2480了外面天天抄顶的现在鸦雀无声屁都不放一个天天让你...
4,2024-07-17 09:55:12,113,这里的评论员没有一个行的根本就没有什么大师都是满嘴放炮黄金一路不回头错过了人生中难得的致富或...
...,...,...,...
284,2024-08-09 23:39:20,101,白银暴跌黄金硬撑白银反弹黄金暴涨资本根据各自散户的持仓基数分别收割
285,2024-08-10 00:42:22,85,都说黄金只能多不能空 不可能下跌黄金才是真正做多赚钱养家糊口
286,2024-08-10 02:23:27,125,黄 金 现 价 2 4 2 8 2 7 附 近 位 可 以 适 当 的 做 多 上 来 美原...
287,2024-08-10 15:14:49,313,金价贵吗全球黄金市值16万亿美元加密货币市值都2 6万亿美元虚拟的不到10年就这么高市值黄金...


In [5]:
s = 'AAA我是分词的间隔符BBB'.join(list(data['summarization']))
#s

In [7]:
m = Leijieba(s,t='./stopwords.txt',\
             b='./userdic.txt')

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\86155\AppData\Local\Temp\jieba.cache
Loading model cost 0.567 seconds.
Prefix dict has been built successfully.


In [25]:
s2 = ' '.join(m.cut1(1,1))
#s2

In [27]:
txtf_news = open('./cnews.jieba1.txt','wt', encoding='utf-8')
txtf_news.write(s2.replace('AAA我是分词的间隔符BBB','\n'))
txtf_news.close()

In [29]:
txt = pd.read_csv('./cnews.jieba1.txt',\
                  skip_blank_lines=False,encoding = 'utf-8', header = None)
outer_join = data.join(txt, how='outer')
outer_join.columns = ['publish_time', 'view_counts	', 'summarization', 'x']

In [31]:
outer_join

,publish_time,view_counts\t,summarization,x
0,2024-07-17 09:27:20,54,黄金建议2477空止损2484目标2465 2457 2450,黄金 建议 2477 止损 目标 2465 2457 2450
1,2024-07-17 09:38:47,163,出不去的资金都没有更好的投资方式黄金与四大行的股票是仅剩的保值方向,资金 投资 黄金 股票 保值 方向
2,2024-07-17 09:45:26,257,昨晚梦中刨土从土里刨出了孙猴子梦醒看一眼时间0219再看一眼黄金走势联想起刚刚的梦境孙悟空空...,昨晚 一眼 时间 一眼 黄金 走势 刚刚 收盘
3,2024-07-17 09:53:46,184,黄金真是给力周一还在2401今天就到2480了外面天天抄顶的现在鸦雀无声屁都不放一个天天让你...,黄金 周一 2401 2480 外面 天天 天天 赶紧
4,2024-07-17 09:55:12,113,这里的评论员没有一个行的根本就没有什么大师都是满嘴放炮黄金一路不回头错过了人生中难得的致富或...,黄金 一路 回头 错过 机会 一路 回调
...,...,...,...,...
284,2024-08-09 23:39:20,101,白银暴跌黄金硬撑白银反弹黄金暴涨资本根据各自散户的持仓基数分别收割,白银 暴跌 黄金 白银 反弹 黄金 暴涨 持仓
285,2024-08-10 00:42:22,85,都说黄金只能多不能空 不可能下跌黄金才是真正做多赚钱养家糊口,黄金 只能 下跌 黄金 赚钱
286,2024-08-10 02:23:27,125,黄 金 现 价 2 4 2 8 2 7 附 近 位 可 以 适 当 的 做 多 上 来 美原...,原油 外汇 黄金 国际 贵金属
287,2024-08-10 15:14:49,313,金价贵吗全球黄金市值16万亿美元加密货币市值都2 6万亿美元虚拟的不到10年就这么高市值黄金...,金价 全球 黄金 市值 万亿美元 货币 市值 万亿美元 不到 市值 黄金


In [33]:
outer_join.to_csv('分词12.csv',index=False)

In [47]:
fenci1 = pd.read_csv('分词3.csv')
fenci1

,publish_time,view_counts\t,summarization,x
0,2014-01-10 13:09:47,653,美国知名矿商Goldcorp周四(1月9日)在年度产量报告中指出2013年黄金产量同比上涨11,美国 知名 周四 年度 产量 报告 指出 2013 黄金 产量 同比 上涨
1,2014-01-10 13:11:48,643,美国黄金矿业公司Golden Star周四(1月9日)公布2013年全年产量报告称该公司预计...,美国 黄金 矿业 公司 周四 公布 2013 产量 报告 公司 预计 成本 此前 预估
2,2014-01-10 13:16:57,598,道明证券 TD Securities 周四(1月9日)发表报告称由于受到美联储进一步缩减货币...,TD Securities 周四 发表 报告 美联储 进一步 缩减 货币基金 利率 QE ...
3,2014-01-10 13:18:03,1122,非农数据公布前夕国际金价交投异常谨慎黄金早间开盘于1233,非农 数据 公布 前夕 国际 金价 交投 异常 谨慎 黄金 早间 开盘 1233
4,2014-01-10 13:53:07,1057,随着黄金价格继续走低 黄金市场也再次走入普通投资者的视野,黄金价格 走低 黄金市场 投资者
...,...,...,...,...
9995,2014-08-15 11:11:45,1143,国际现货金价周四(8月14日)在清淡交投中小涨但涨幅受限因美国初请失业金人口数据显示就业市场...,国际 现货 金价 周四 清淡 交投 涨幅 受限 美国 初请 失业 数据 显示 就业 市场 ...
9996,2014-08-15 11:25:48,610,黑色论金投资黄金的十大优点,黑色 论金 投资 黄金 十大 优点
9997,2014-08-15 11:33:02,863,"黄金价格继续下跌 全球股市周四(8月15日)继续收盘,分析人士认为黄金的价格将在今年年底跌至...",黄金价格 下跌 全球股市 周四 收盘 分析 人士 黄金 价格 跌至 9700 美元
9998,2014-08-15 11:35:18,736,Kitco特约分析师 Peter Hug认为目前黄金市场基本面是看空的而实物需求基本没有,Kitco 分析师 黄金市场 基本面 看空 实物 需求


In [ ]:
result = pd.concat([df1, df2], ignore_index=True)

In [3]:
import pandas as pd
data1 = pd.read_csv('./分词1.csv')
data2 = pd.read_csv('./分词2.csv')
data3 = pd.read_csv('./分词3.csv')
data4 = pd.read_csv('./分词4.csv')
data5 = pd.read_csv('./分词5.csv')
data6 = pd.read_csv('./分词6.csv')
data7 = pd.read_csv('./分词7.csv')
data8 = pd.read_csv('./分词8.csv')
data9 = pd.read_csv('./分词9.csv')
data10 = pd.read_csv('./分词10.csv')
data11 = pd.read_csv('./分词11.csv')
data12 = pd.read_csv('./分词12.csv')

In [25]:
file11 = pd.concat([file10, data12], ignore_index=True)

In [29]:
file11.to_csv('./分词过的.csv',index = False)